# Regresión logística

## Carga y análisis de los datos

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/JASDataCTG/Diplomado-ML/main/Modulo%202/Datasets/casas.csv', header = 0)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sns.boxplot(y = 'n_cam_hos', data = df)

In [ ]:
sns.boxplot(y = df['n_hab_hotel'])

In [ ]:
sns.jointplot(x = df['lluvia_ano'], y = df['Vendida'])

In [ ]:
sns.countplot(x = df['aeropuerto'])

In [ ]:
df.groupby(by = 'Vendida')['area_resid'].mean()

In [ ]:
sns.countplot(y = df['cuerpo_agua'])

In [ ]:
sns.countplot(x = df['ter_bus'])

In [ ]:
x1 = df.precio[df.Vendida == 1]
x2 = df.precio[df.Vendida == 0]#df[df.Vendida == 0]['precio']

kwargs = dict(alpha = 0.5, bins = 8, density = True, stacked = True)

plt.hist(x1, **kwargs, color = 'g', label = 'Vendida')
plt.hist(x2, **kwargs, color = 'b', label = 'No vendida')
plt.legend()

## Tratamiento de valores extremos

In [ ]:
np.percentile(df.n_hab_hotel, [99])[0]

In [ ]:
lim_sup = np.percentile(df.n_hab_hotel, [99])[0]

In [ ]:
df[df.n_hab_hotel > lim_sup]

In [ ]:
df.n_hab_hotel[df.n_hab_hotel > 3 * lim_sup] = 3 * lim_sup

In [ ]:
np.percentile(df.lluvia_ano, [0.3])[0]

In [ ]:
lim_inf = np.percentile(df.lluvia_ano, [0.3])[0]

In [ ]:
df[df.lluvia_ano < lim_inf]

In [ ]:
df.lluvia_ano[df.lluvia_ano < 0.3 * lim_inf] = 0.3 * lim_inf

In [ ]:
df.describe()

In [ ]:
df.n_cam_hos = df.n_cam_hos.fillna(df.n_cam_hos.mean())

In [ ]:
df.info()

In [ ]:
df['prom_dist'] = (df.dist1 + df.dist2 + df.dist3 + df.dist4) / 4

In [ ]:
del df['dist1']

In [ ]:
del df['dist2']

In [ ]:
del df['dist3']

In [ ]:
del df['dist4']

In [ ]:
del df['ter_bus']

In [ ]:
df = pd.get_dummies(df)

In [ ]:
df.head()

In [ ]:
del df['aeropuerto_NO']

In [ ]:
del df['cuerpo_agua_Ninguno']

In [ ]:
df.head()

In [ ]:
df.to_csv('DataReg.csv', header = True, index = False)

## Regresión logística
Dado que la función lineal no se puede ajustar a variables catagóricas binarias (0, 1), se hace necesario utilizar la función sigmoide $\sigma(y)$:

$\large{\sigma(y)=\frac{1}{1+e^{-y}}}$

Cuando $y$ toma valores positivos muy grandes, $e^{-y}$ toma volores muy cercanos a cero, lo que ocasiona que la función $\sigma(y)$ tome valores muy cercanos a $1$. Por el contrario, cuando $y$ toma valores negativos muy grandes, $e^{-y}$ toma volores muy cercanos a $+\infty$, lo que ocasiona que la función $\sigma(y)$ tome valores muy cercanos a $0$

Probabilidad de $y$ dado $X$:

$\large{P(y=1|X=x)=\frac{e^{\theta_{0}+\theta_{1}x_{1}+\theta_{2}x_{2}+\theta_{3}x_{3}+...+\theta_{p}x_{p}}}{1+e^{-(\theta_{0}+\theta_{1}x_{1}+\theta_{2}x_{2}+\theta_{3}x_{3}+...+\theta_{p}x_{p})}}}$

La regresión logística calcula los logaritmos de las probabilidades de cada variable para transformarlas en 0 o 1, usando un umbral de decisión.

In [ ]:
# Se utiliza doble corchete porque para el modelo X debe ser un dataframe y por defecto una sola columna es un
# tipo de dato Serie
X = df[['precio']]

In [ ]:
X.head()

In [ ]:
# El modelo requiere la variable predicha como una serie, por eso no se aplican dobles corchetes
y = df['Vendida']

In [ ]:
y.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import accuracy_score, precision_score, confusion_matrix

In [ ]:
# Crear modelo de regresión logística
reg_log_simple = LogisticRegression()

In [ ]:
# Entrenar el modelo
reg_log_simple.fit(X, y)

In [ ]:
# Obener el valor de theta(1)
reg_log_simple.coef_

In [ ]:
# Obtener el valor de theta(0)
reg_log_simple.intercept_

In [ ]:
# Importar el modulo statsmodel para poder crear la constante que requiere el modelo Logit
import statsmodels.api as sn

In [ ]:
const_X = sn.add_constant(X)

In [ ]:
const_X

In [ ]:
# Importar el modulo para trabajar con variables discretas con Logit
import statsmodels.discrete.discrete_model as sm

In [ ]:
# Entrenar y ajustar el modelo a los datos de entrada
logit = sm.Logit(y, const_X).fit()

In [ ]:
# Obtener el resumen del modelo
logit.summary() 

In [ ]:
# Realizar una predicción de ejemplo
np.floor(logit.predict([1,-300])[0])

## Gráfica del modelo de regresión logística simple

In [ ]:
# Crear los puntos necesarios de la variable predicha desarrollar la gráfica

grilla_X = np.linspace(
    start = -100, # min(df.precio),
    stop = 100, # max(df.precio),
    num = 500
).reshape(-1,1)

# Agregar la constante para poder desarrollar la predicción por el modelo Logit
grilla_X = sn.add_constant(grilla_X, prepend = True)

# Desarrollar la predicción
prediccion = logit.predict(exog = grilla_X)

In [ ]:
grilla_X.shape # El resultado es un vector de 500 x 2

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
# Nube de puntos para cuando la propiedad es vendida
ax.scatter(
    x = df[['precio']][(y==1)].to_numpy().flatten(), y = df[['Vendida']][y==1].to_numpy().flatten()
)
# Nube de puntos para el caso en que la propiedad no es vendida
ax.scatter(
    x = df[['precio']][(y==0)].to_numpy().flatten(), y = df[['Vendida']][y==0].to_numpy().flatten()
)

ax.plot(grilla_X[:500, 1], prediccion, color = "gray") # Segunda columna de la grilla
ax.set_title("Modelo regresión logística")
ax.set_ylabel("P(Vendida = 1 | precio)")
ax.set_xlabel("Precio de venta")

In [ ]:
prediccion